In [3]:
library(Seurat)
library(tidyverse)
library("SCINA")
library(future)
library(clustree)

Warning message:
"le package 'Seurat' a été compilé avec la version R 4.2.3"


The legacy packages maptools, rgdal, and rgeos, underpinning this package
will retire shortly. Please refer to R-spatial evolution reports on
https://r-spatial.org/r/2023/05/15/evolution4.html for details.
This package is now running under evolution status 0 

Attaching SeuratObject


Attachement du package : 'Seurat'


L'objet suivant est masqué depuis 'package:SummarizedExperiment':

    Assays


Warning message:
"le package 'tidyverse' a été compilé avec la version R 4.2.3"
Warning message:
"le package 'ggplot2' a été compilé avec la version R 4.2.3"
Warning message:
"le package 'tibble' a été compilé avec la version R 4.2.3"
Warning message:
"le package 'tidyr' a été compilé avec la version R 4.2.3"
Warning message:
"le package 'readr' a été compilé avec la version R 4.2.3"
Warning message:
"le package 'purrr' a été compilé avec la version R 4.2.3"
Warning message:
"le package 'dplyr' a été compilé avec la version R 4.2.3"
Warning message:
"le package 'stringr' a été compilé avec l

In [2]:
#devtools::install_github("dynverse/dyno")
library(dyno)

Le chargement a nécessité le package : dynfeature

Le chargement a nécessité le package : dynguidelines

Le chargement a nécessité le package : dynmethods

Le chargement a nécessité le package : dynplot


Attachement du package : 'dynplot'


L'objet suivant est masqué depuis 'package:ggraph':

    theme_graph


Le chargement a nécessité le package : dynwrap



In [8]:
seurat_obj<- readRDS(file = "./html_lines/Norm_BT54_diff.rds")
seurat_obj

An object of class Seurat 
49074 features across 979 samples within 2 assays 
Active assay: SCT (22502 features, 3000 variable features)
 1 other assay present: RNA

In [4]:
dataset <- wrap_expression(
  counts = seurat_obj@assays$SCT@counts,
  expression = seurat_obj@assays$SCT@data
)

In [1]:
library(monocle3)

Le chargement a nécessité le package : Biobase

Le chargement a nécessité le package : BiocGenerics


Attachement du package : 'BiocGenerics'


Les objets suivants sont masqués depuis 'package:stats':

    IQR, mad, sd, var, xtabs


Les objets suivants sont masqués depuis 'package:base':

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', and for packages 'citation("pkgname")'.


Le chargement a nécessité le package : SingleCellExperiment

Le chargement a nécessité le package : Summarize

In [9]:
## convert seurat to cell_data_set obj
cds <- as.cell_data_set(seurat_obj)

In [13]:
fData(cds)$gene_name <- rownames(fData(cds))
fData(cds)

DataFrame with 22502 rows and 1 column
                      gene_name
                    <character>
ATAD3B                   ATAD3B
PRDM16                   PRDM16
PEX10                     PEX10
PEX14                     PEX14
PLCH2                     PLCH2
...                         ...
ENSG00000273748 ENSG00000273748
ENSG00000278817 ENSG00000278817
ENSG00000278384 ENSG00000278384
ENSG00000276345 ENSG00000276345
ENSG00000271254 ENSG00000271254

In [ ]:
## use seurat clusters

## assign partitions
recreate.partition <- c(rep(1, length(cds@colData@rownames)))
names(recreate.partition) <- cds@colData@rownames
recreate.partition <- as.factor(recreate.partition)

cds@clusters$UMAP$partitions <- recreate.partition

In [ ]:
## assign cluster infos
cds@clusters$UMAP$clusters <- seurat_obj$seurat_clusters

In [ ]:
## assign UMAP coordinate - cell embeddings
cds@int_colData@listData$reduceDims$UMAP <- seurat_obj@reductions$umap@cell.embeddings

In [ ]:
# plot

cluster.before.trajectory <- plot_cells(cds,
           color_cells_by = 'cluster',
           label_groups_by_cluster = FALSE,
           group_label_size = 5) +
  theme(legend.position = "right")

cluster.names <- plot_cells(cds,
           color_cells_by = "redefined_cluster",
           label_groups_by_cluster = FALSE,
           group_label_size = 5) +
  #scale_color_manual(values = c('red', 'blue', 'green', 'maroon', 'yellow', 'grey', 'cyan')) +
  theme(legend.position = "right")

cluster.before.trajectory | cluster.names


In [ ]:
# Learn 

### Seurat object (all genes)

In [3]:
obj_all <-  readRDS(file = "ALL_mca.rds")
obj_all

An object of class Seurat 
36334 features across 9840 samples within 1 assay 
Active assay: RNA (36334 features, 2000 variable features)
 4 dimensional reductions calculated: pca, umap, tsne, mca

In [2]:
obj.combined <- readRDS(file = "./results/ALL.rds")
obj.combined

# 36334 features across 9840 samples within 1 assay 

An object of class Seurat 
36334 features across 9840 samples within 1 assay 
Active assay: RNA (36334 features, 2000 variable features)
 3 dimensional reductions calculated: pca, umap, tsne

### Seurat object (metabo only)

In [17]:
metabo.object <- readRDS(file = "./results/ALL_metabo.rds")
metabo.object
# 2682 features across 9840 samples within 1 assay 

An object of class Seurat 
2682 features across 9840 samples within 1 assay 
Active assay: RNA (2682 features, 1000 variable features)
 3 dimensional reductions calculated: pca, umap, tsne

# Monocle3

In [15]:
#devtools::install_github('cole-trapnell-lab/monocle3')
library(monocle3)
#devtools::install_github("satijalab/seurat-wrappers")
library(SeuratWrappers)

deldir       (1.0-6   -> 1.0-9  ) [CRAN]
spatstat.... (3.0-2   -> 3.0-3  ) [CRAN]
spatstat.... (3.1-4   -> 3.1-5  ) [CRAN]
spatstat.... (3.1-0   -> 3.2-1  ) [CRAN]
fs           (1.6.1   -> 1.6.2  ) [CRAN]
cachem       (1.0.7   -> 1.0.8  ) [CRAN]
rlang        (1.1.0   -> 1.1.1  ) [CRAN]
later        (1.3.0   -> 1.3.1  ) [CRAN]
jsonlite     (1.8.4   -> 1.8.5  ) [CRAN]
httpuv       (1.6.9   -> 1.6.11 ) [CRAN]
future.apply (1.10.0  -> 1.11.0 ) [CRAN]
sp           (1.6-0   -> 1.6-1  ) [CRAN]
vctrs        (0.6.1   -> 0.6.2  ) [CRAN]
matrixStats  (0.63.0  -> 1.0.0  ) [CRAN]
dplyr        (1.1.1   -> 1.1.2  ) [CRAN]
curl         (5.0.0   -> 5.0.1  ) [CRAN]
igraph       (1.4.2   -> 1.4.3  ) [CRAN]
reticulate   (1.28    -> 1.30   ) [CRAN]
spatstat.... (3.1-0   -> 3.2-1  ) [CRAN]
scattermore  (0.8     -> 1.2    ) [CRAN]
BiocManager  (1.30.20 -> 1.30.21) [CRAN]


Installing 21 packages: deldir, spatstat.utils, spatstat.random, spatstat.geom, fs, cachem, rlang, later, jsonlite, httpuv, future.apply, sp, vctrs, matrixStats, dplyr, curl, igraph, reticulate, spatstat.explore, scattermore, BiocManager

Warning message:
"les packages 'matrixStats', 'dplyr' sont en cours d'utilisation et ne seront pas installés"
Installation des packages dans 'C:/Users/Soeur/AppData/Local/R/win-library/4.2'
(car 'lib' n'est pas spécifié)




  Une version binaire est disponible mais la version du source est plus
  récente:
            binary source needs_compilation
scattermore    1.1    1.2              TRUE

le package 'deldir' a été décompressé et les sommes MD5 ont été vérifiées avec succés


Warning message:
"impossible de supprimer l'installation précédente du package 'deldir'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problème lors de la copie de C:\Users\Soeur\AppData\Local\R\win-library\4.2\00LOCK\deldir\libs\x64\deldir.dll vers C:\Users\Soeur\AppData\Local\R\win-library\4.2\deldir\libs\x64\deldir.dll: Permission denied"
Warning message:
"'deldir' restauré"


le package 'spatstat.utils' a été décompressé et les sommes MD5 ont été vérifiées avec succés


...

Warning message:
"impossible de supprimer l'installation précédente du package 'spatstat.explore'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problème lors de la copie de C:\Users\Soeur\AppData\Local\R\win-library\4.2\00LOCK\spatstat.explore\libs\x64\spatstat.explore.dll vers C:\Users\Soeur\AppData\Local\R\win-library\4.2\spatstat.explore\libs\x64\spatstat.explore.dll: Permission denied"
Warning message:
"'spatstat.explore' restauré"


le package 'BiocManager' a été décompressé et les sommes MD5 ont été vérifiées avec succés

Les packages binaires téléchargés sont dans
	C:\Users\Soeur\AppData\Local\Temp\RtmpisaJNZ\downloaded_packages


installation du package source 'scattermore'


Warning message in i.p(...):
"l'installation du package 'scattermore' a eu un statut de sortie non nul"


── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file 'C:\Users\Soeur\AppData\Local\Temp\RtmpisaJNZ\remotes44dc17157642\satijalab-seurat-wrappers-d28512f/DESCRIPTION' ... OK
* preparing 'SeuratWrappers':
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
Omitted 'LazyData' from DESCRIPTION
* building 'SeuratWrappers_0.3.1.tar.gz'



Installation du package dans 'C:/Users/Soeur/AppData/Local/R/win-library/4.2'
(car 'lib' n'est pas spécifié)



In [19]:
monocle <- as.cell_data_set(metabo.object)
monocle

Warning message:
"Monocle 3 trajectories require cluster partitions, which Seurat does not calculate. Please run 'cluster_cells' on your cell_data_set object"


class: cell_data_set 
dim: 2682 9840 
metadata(0):
assays(3): counts logcounts scaledata
rownames(2682): SLC2A5 NMNAT1 ... TKTL1 PGAM4
rowData names(0):
colnames(9840): 225diff_AAACCCAAGAGCTGAC 225diff_AAACGAAGTAGACAGC ...
  336_TTTGTTGCAAGGCCTC 336_TTTGTTGCACCGCTAG
colData names(15): orig.ident nCount_RNA ... ident Size_Factor
reducedDimNames(3): PCA UMAP TSNE
mainExpName: RNA
altExpNames(0):

In [20]:
cell_metadata <- colData(monocle)
fData(monocle)$gene_name <- rownames(fData(monocle))

In [26]:
monocle$colData

NULL

In [22]:
recreate_partition <- c(rep(1,length(monocle$colData@rownames)))
names(recreate_partition) <- monocle@colData@rownames
recreate_partition <- as.factor(recreate_partition)

ERROR: Error in eval(expr, envir, enclos): tentative d'obtenir le slot "rownames" d'un objet d'une classe élémentaire ("NULL") sans slots
